In [ ]:
!pip install datasets

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from sklearn.preprocessing import LabelEncoder
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install huggingface_hub

In [ ]:
# 1. Авторизация HF
!huggingface-cli login

In [ ]:
# Загрузить новый датасет из HF
dataset = load_dataset("takiholadi/kill-me-please-dataset")

In [ ]:
# Разделение на обучающую и тестовую выборки
train_dataset = dataset['train'].remove_columns(['votes', 'url', 'datetime'])
val_dataset = dataset['test'].remove_columns(['votes', 'url', 'datetime'])

In [ ]:
# Просмотр
train_dataset = pd.DataFrame(train_dataset)
print(train_dataset.head(10))

In [ ]:
# Проверка наличия None значений в столбце 'tags'
print("Количество None значений в train_dataset['tags']:", train_dataset['tags'].isna().sum())
val_dataset = pd.DataFrame(val_dataset)
print("Количество None значений в val_dataset['tags']:", val_dataset['tags'].isna().sum())

In [ ]:
train_dataset = train_dataset.dropna(subset=['tags'])

In [ ]:
# Перенумерация меток, чтобы они были последовательными
unique_labels = sorted(set(train_dataset['tags']) | set(val_dataset['tags']))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
train_labels = [label_mapping[label] for label in train_dataset['tags']]
val_labels = [label_mapping[label] for label in val_dataset['tags']]
num_labels = len(unique_labels)

In [ ]:
# Предпросмотр новых меток
print("Новые метки после перекодирования:", label_mapping)

In [ ]:
# Сохранение словаря в файл для дальнейшего использования
with open('label_mapping.json', 'w') as f:
    json.dump(label_mapping, f)

In [ ]:
# 3. Загрузка токенизатора и модели
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = BertForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', num_labels=num_labels)

In [ ]:
# 4. Токенизация данных
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True, max_length=128)

In [ ]:
# 5. Создание PyTorch Dataset
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # Конструктор класса, принимает токенизированные данные и метки
        self.encodings = encodings  # Сохранение токенизированных данных в объекте
        self.labels = labels  # Сохранение меток в объекте

    def __getitem__(self, idx):
        # Возвращает один элемент данных по индексу
        # Создает словарь, в котором ключи - это поля токенизированных данных, а значения - соответствующие тензоры
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])  # Добавление метки класса в элемент
        return item

    def __len__(self):
        # Возвращает количество элементов в датасете
        return len(self.labels)

In [ ]:
train_dataset = TextDataset(train_encodings, train_labels)  # Создание объекта TextDataset для обучающей выборки
val_dataset = TextDataset(val_encodings, val_labels)  # Создание объекта TextDataset для тестовой выборки

In [ ]:
# 6. Настройка параметров обучения
training_args = TrainingArguments(
    output_dir='./results',          # выходная директория
    num_train_epochs=3,              # количество эпох
    per_device_train_batch_size=16,  # размер batch для обучения
    per_device_eval_batch_size=16,   # размер batch для оценки
    evaluation_strategy='epoch',     # стратегия оценки
    save_strategy='epoch',           # стратегия сохранения модели
    logging_dir='./logs',            # директория для логов
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

In [ ]:
# 7. Определение метрик для оценки
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return {'accuracy': acc, 'f1': f1}

In [ ]:
# 8. Создание объекта Trainer и запуск обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
# 9. Сохранение модели
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

In [ ]:
# 10. Проверка работы модели - генерация текстов и предсказание
sample_texts = [
    "Сегодня у меня был ужасный день на работе, все пошло не так.",
    "Пошел на прогулку с друзьями, чувствую себя счастливым.",
    "Сломался мой ноутбук, не знаю, что делать."
    "Сдал экзамен на отлично, очень рад своему успеху!",
    "Поссорился с коллегой на работе, чувствую себя подавленным.",
    "Записался на курсы по йоге, надеюсь улучшить свое здоровье и настроение."
]

In [ ]:
# Токенизация тестовых текстов
sample_encodings = tokenizer(sample_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

In [ ]:
# Определение устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Перевод модели на устройство и в режим оценки
model.to(device)
model.eval()

In [ ]:
# Перемещение входных данных на то же устройство
sample_encodings = {key: val.to(device) for key, val in sample_encodings.items()}

In [ ]:
# Прогон текстов через модель и получение предсказаний
with torch.no_grad():
    outputs = model(**sample_encodings)
    predictions = outputs.logits.argmax(dim=-1)

In [ ]:
# Расшифровка предсказанных меток
with open('label_mapping.json', 'r') as f:
    label_mapping = json.load(f)
inv_label_mapping = {v: k for k, v in label_mapping.items()}
predicted_labels = [inv_label_mapping[int(pred)] for pred in predictions]

In [ ]:
# Вывод предсказаний
for text, label in zip(sample_texts, predicted_labels):
    print(f"Текст: {text}\nПредсказанные теги: {label}\n")